In [1]:
import os
import pandas as pd
import numpy as np

import re # regular expressions 

%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool

output_notebook()

# data source - US Federal Emergency Management Agency
# https://www.fema.gov/media-library/assets/documents/28318

Loading BokehJS ...

In [2]:
# get data to analyze

# Option 1 - from S3
# set up link to S3 bucket where out data is
# if running this notebook locally on Windows; you must comment out this section; as AWS has deprecated sagemaker on Win
# import boto3
# import pandas as pd
# from sagemaker import get_execution_role
# role = get_execution_role()

# replace the text inside single quotes with your S3 bucket name
bucket='sagemaker-setup-demo-pragamaticai-00'

# replace the text inside single quotes with your file name in S3 that has your data
data_key = 'data.gov.FEMADeclarations.12.3.18.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

# read data file
# df = pd.read_csv(data_location, sep=',')

# Option 2 - from local excel file
# comment out line 18 `df = pd.read_csv...` above, and uncomment pd.read_csv or pd.read_excel below
df = pd.read_excel("data.gov.FEMADeclarations.12.3.18.xlsx", sheet_name="FEMA Declarations", skiprows=3)

# show sample of file we just read
df.head(3)

,Disaster Number,IH Program Declared,IA Program Declared,PA Program Declared,HM Program Declared,State,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,Disaster Close Out Date,Place Code,Declared County/Area,Declaration Request Number
0,3410,No,No,Yes,No,AK,2018-11-30 18:15:00,EM,Earthquake,EARTHQUAKE,2018-11-30,2018-11-30 23:59:00,NaT,99020.0,Anchorage (Borough),18133
1,3410,No,No,Yes,No,AK,2018-11-30 18:15:00,EM,Earthquake,EARTHQUAKE,2018-11-30,2018-11-30 23:59:00,NaT,99122.0,Kenai Peninsula (Borough),18133
2,3410,No,No,Yes,No,AK,2018-11-30 18:15:00,EM,Earthquake,EARTHQUAKE,2018-11-30,2018-11-30 23:59:00,NaT,99170.0,Matanuska-Susitna (Borough),18133


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48427 entries, 0 to 48426
Data columns (total 16 columns):
Disaster Number               48427 non-null int64
IH Program Declared           48427 non-null object
IA Program Declared           48427 non-null object
PA Program Declared           48427 non-null object
HM Program Declared           48427 non-null object
State                         48427 non-null object
Declaration Date              48427 non-null datetime64[ns]
Disaster Type                 48427 non-null object
Incident Type                 48427 non-null object
Title                         48427 non-null object
Incident Begin Date           48427 non-null datetime64[ns]
Incident End Date             48088 non-null datetime64[ns]
Disaster Close Out Date       38724 non-null datetime64[ns]
Place Code                    48230 non-null float64
Declared County/Area          48230 non-null object
Declaration Request Number    48427 non-null int64
dtypes: datetime64[ns](4), f

In [4]:
df.dropna(axis=0, how='any', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38190 entries, 1054 to 48337
Data columns (total 16 columns):
Disaster Number               38190 non-null int64
IH Program Declared           38190 non-null object
IA Program Declared           38190 non-null object
PA Program Declared           38190 non-null object
HM Program Declared           38190 non-null object
State                         38190 non-null object
Declaration Date              38190 non-null datetime64[ns]
Disaster Type                 38190 non-null object
Incident Type                 38190 non-null object
Title                         38190 non-null object
Incident Begin Date           38190 non-null datetime64[ns]
Incident End Date             38190 non-null datetime64[ns]
Disaster Close Out Date       38190 non-null datetime64[ns]
Place Code                    38190 non-null float64
Declared County/Area          38190 non-null object
Declaration Request Number    38190 non-null int64
dtypes: datetime64[ns](4)

In [5]:
df['Declaration Date'].unique()

array(['2017-09-18T12:30:00.000000000', '2017-09-11T17:00:00.000000000',
       '2017-09-08T22:15:00.000000000', ...,
       '1964-12-29T00:00:00.000000000', '1964-12-24T00:00:00.000000000',
       '1959-01-29T00:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
# create months column, convert key columns to int type, and sort
# df['monthDeclared'] = 
df.insert(7, 'fyDeclared', df['Declaration Date'].str.split('/').str[2])
df["fyDeclared"] = pd.to_numeric(df["fyDeclared"])
df.insert(8, 'monthDeclared', df['Declaration Date'].str.split('/').str[0])
df["monthDeclared"] = pd.to_numeric(df["monthDeclared"])
df.info()

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [ ]:
df.head(5)

In [ ]:
df.sort_values(by=['fyDeclared', 'monthDeclared'], ascending=True, inplace=True)
df.head(5)

In [ ]:
# what kind of incidents exist in this data?
df['Title'] = df.Title.str.title()
df_count = pd.DataFrame(df['Title'].unique())
df_count

In [ ]:
# we are interested in fires
# drop all rows that are not Fires
df = df[pd.Series(df['Title']).str.contains('Fire')]
df.info()

In [ ]:
df.fyDeclared.unique()

In [ ]:
months = sorted(df.monthDeclared.unique());

In [ ]:
sns.distplot(df.fyDeclared);

In [ ]:
# bokeh notebook

from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.sampledata.commits import data
from bokeh.transform import jitter

output_notebook()

MONTHS = [str(i) for i in months]

source = ColumnDataSource(df)

plot = figure(plot_width=950, plot_height=300, y_range=MONTHS, x_axis_type='linear',
           title="US Fire Incidents 1953-2020")

plot.circle(x='fyDeclared', y=jitter('monthDeclared', width=0.6, range=plot.y_range),  source=source, alpha=0.3)
plot.xaxis.axis_label = 'Year'
plot.xaxis.axis_label_text_font_style = "bold"
plot.yaxis.axis_label = 'Month #'
plot.yaxis.axis_label_text_font_style = "bold"

plot.ygrid.grid_line_color = None

show(plot);

In [ ]:
df_summary = df.groupby(['fyDeclared']).size().reset_index(name="annual_#_of_fires")

In [ ]:
# bokeh notebook

x, y = df_summary['fyDeclared'], df_summary['annual_#_of_fires']

# Basic plot setup
plot = figure(plot_width=900, plot_height=500,
              x_axis_type="linear", tools="pan,xzoom_in,reset",
              toolbar_location="below", 
              title='Annual # of Wildfires:  1953-2018')

plot.background_fill_color = "black"
plot.background_fill_alpha = 0.005

plot.line(x, y, line_dash="4 4", line_width=1, color='gray')
plot.xaxis.axis_label = 'Year'
plot.xaxis.axis_label_text_font_style = "bold"
plot.yaxis.axis_label = '# of Fires'
plot.yaxis.axis_label_text_font_style = "bold"

cr = plot.circle(x, y, size=25,
                fill_color="grey", hover_fill_color="firebrick",
                fill_alpha=0.15, hover_alpha=0.3,
                line_color=None, hover_line_color="white")

source = ColumnDataSource(df_summary)
TOOLTIPS = [("DATE", '$fyDeclared')]
plot.add_tools(HoverTool(renderers=[cr], mode='vline'))

show(plot)


In [ ]:
df_summary_months = df.groupby(['fyDeclared', 'monthDeclared']).size().reset_index(name="monthly_fires_by_year")
df_summary_months['fires_this_year'] = df.groupby(['fyDeclared'])['fyDeclared'].agg('sum')
df_summary_months